In [1]:
import pathlib
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Polygon, LineString, Point
import math
import warnings
warnings.filterwarnings(action='ignore')
#import descartes
import folium
from tqdm.notebook import tqdm

In [2]:
%cd E:\yujin\INHA\data

E:\yujin\INHA\data


In [3]:
owner_land = gpd.read_file('14.광양시_소유지정보.geojson')
gyboundary = gpd.read_file('20.광양시_행정경계(읍면동).geojson')

parkingLot_raw = pd.read_csv('02.광양시_주차장_공간정보.csv')
traf_parkingLot_raw = gpd.read_file('26.광양시_도시계획(교통시설).geojson')
campsite_raw = pd.read_csv('05.광양시_대중집합시설_야영장.csv')
CulPys_raw = gpd.read_file('25.광양시_도시계획(공공문화체육시설).geojson')
chargingStation_raw = pd.read_csv('01.광양시_충전기설치현황.csv')
building_raw = pd.read_csv('광양시_건물정보.csv')

#### 면적과 소유자 정보계산은 시간이 오래 걸리기 때문에 원래의 데이터에 계산한 컬럼을 추가해놓고 앞으로는 컬럼이 추가된 데이터를 사용한다.

# 1. 야영장
1) lon, lat가지고 geometry컬럼 만들기  
2) '14.광양시_소유지정보.geojson'데이터를 바탕으로 야영지 면적을 구하기 (컬럼명 : DGM_AR)  
3) cateogory컬럼 추가  
4) 컬럼명 변경 (소유주체(공공/민간) --> 소유주체)  
5) 주차면수 컬럼 추가

In [27]:
campsite_raw['geometry'] = campsite_raw.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
owner_list= np.array([owner_land.토지면적[j]  for i in range(len(campsite_raw)) for j in range(len(owner_land)) if campsite_raw.geometry[i].within(owner_land.geometry[j])==True ])
campsite_raw['DGM_AR']=owner_list
campsite_raw['category'] = '야영장'
campsite_raw.rename(columns={"소유주체(공공/민간)":"소유주체"}, inplace=True)
campsite_raw['주차면수'] = (campsite_raw['DGM_AR']/300).astype(int)

campsite_raw

,명칭,주소,소유주체,lon,lat,geometry,DGM_AR,category,주차면수
0,진틀펜션야영장,광양시 옥룡면 신재로 1678-54,민간,127.601787,35.096566,POINT (127.601787 35.096566),597.0,야영장,1
1,느랭이골자연휴양림,광양시 다압면 토끼재길 119-32,민간,127.710849,35.060546,POINT (127.710849 35.060546),52236.0,야영장,174
2,백운산수련관야영장,광양시 옥룡면 약수제단길 154,민간,127.631555,35.069553,POINT (127.631555 35.06955300000001),425821.0,야영장,1419
3,봉강하조오토캠핑장,광양시 봉강면 성불로 1097,민간,127.562487,35.076193,POINT (127.562487 35.076193),650.0,야영장,2
4,백운산자연휴양림,광양시 옥룡면 백계로 337,공공,127.591184,35.054216,POINT (127.591184 35.054216),3293527.0,야영장,10978
5,백운제오토캠핑장,광양시 봉강면 봉당리 178,공공,127.581644,35.005226,POINT (127.581644 35.005226),22502.0,야영장,75


In [29]:
campsite_raw.to_csv('광양시_대중집합시설_야영장_전처리.csv', index=False, encoding='cp949')

# 2. 주차장
1) lon, lat가지고 geometry만들기  
2) 컬럼명 변경 (면적 --> DGM_AR)  
3) cateogory 컬럼 추가  
4) 소유지 컬럼 추가  

In [23]:
parkingLot_raw['geometry'] = parkingLot_raw.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
parkingLot_raw.rename(columns={"면적":"DGM_AR"}, inplace=True)
parkingLot_raw['category'] = '주차장'
parkingLot_owner_list= np.array([owner_land.소유구분명[j] for i in range(len(parkingLot_raw.geometry)) for j in range(len(owner_land.geometry)) if parkingLot_raw.geometry[i].within(owner_land.geometry[j])==True])
parkingLot_raw['소유주체'] = parkingLot_owner_list
parkingLot_raw

,주차장명칭,구분,DGM_AR,주차면수,유료/무료,주소,lon,lat,geometry,category,소유주체
0,인동숲앞 주차장,노외,4189.0,161.0,유료,인서리 237-1,127.586883,34.970324,POINT (127.5868828 34.97032367),주차장,군유지
1,구터미널주차장,노외,2968.0,125.0,유료,중동 1651,127.695428,34.936252,POINT (127.6954279 34.93625189999999),주차장,군유지
2,구경찰서 주차장,노외,2836.0,89.0,무료,읍내리 227-1,127.584586,34.975637,POINT (127.5845858 34.97563677),주차장,군유지
3,칠성1지구,노외,1980.0,52.0,무료,칠성리 962-1,127.580683,34.970130,POINT (127.5806828 34.97013011),주차장,군유지
4,칠성1지구,노외,1970.0,64.0,무료,칠성리 948-1,127.580707,34.972523,POINT (127.5807067 34.97252282),주차장,군유지
...,...,...,...,...,...,...,...,...,...,...,...
230,포석정위도로변,피서지,264.0,10.0,무료,동곡리 662-1,127.604822,35.079214,POINT (127.6048219 35.07921402),주차장,군유지
231,동곡산장옆,피서지,1212.0,60.0,무료,동곡리 549,127.613417,35.070673,POINT (127.6134168 35.07067277),주차장,군유지
232,하천첨주변,피서지,2381.0,96.0,무료,산남리 154-4,127.611774,35.002759,POINT (127.6117741 35.00275939),주차장,개인
233,회계분교,피서지,2090.0,66.0,무료,진상 어치 1140,127.674719,35.092388,POINT (127.6747186 35.09238804),주차장,군유지


In [26]:
parkingLot_raw.to_csv('광양시_주차장_공간정보_전처리.csv', index=False, encoding='cp949')

# 3. 도시계획(교통시설)
1) '주차'관련된 데이터만 추출  
2) geometry의 Multipoligon형태를 Point로 변경  
3) category 컬럼 추가  
4) 소유자 컬럼 추가

In [4]:
traf_parkingLot_raw = gpd.read_file('26.광양시_도시계획(교통시설).geojson')
traf_parkingLot_raw

,PRESENT_SN,DGM_NM,DGM_AR,DGM_LT,geometry
0,46230UQ152PS201202010003,기타 주차장시설,344,174,"MULTIPOLYGON (((127.58420 34.97748, 127.58422 ..."
1,46230UQ152PS201202010002,기타 주차장시설,483,94,"MULTIPOLYGON (((127.58367 34.97843, 127.58386 ..."
2,46230UQ152PS201202010109,노외주차장,1612,167,"MULTIPOLYGON (((127.75156 34.94550, 127.75152 ..."
3,46230UQ152PS201202010106,노외주차장,305,70,"MULTIPOLYGON (((127.70281 34.98895, 127.70281 ..."
4,46230UQ152PS201202010105,기타 주차장시설,4608,286,"MULTIPOLYGON (((127.69026 34.93717, 127.68925 ..."
...,...,...,...,...,...
223,46230UQ152PS201202010143,기타 주차장시설,1244,144,"MULTIPOLYGON (((127.66634 34.93260, 127.66659 ..."
224,46230UQ152PS201202010139,노외주차장,2657,208,"MULTIPOLYGON (((127.58429 34.97586, 127.58433 ..."
225,46230UQ152PS201202010136,일반철도,457315,24470,"MULTIPOLYGON (((127.68957 34.93050, 127.68966 ..."
226,46230UQ152PS201202010130,노외주차장,1313,142,"MULTIPOLYGON (((127.69841 34.92691, 127.69876 ..."


In [5]:
traf_parkingLot_raw=traf_parkingLot_raw[traf_parkingLot_raw['DGM_NM'].str.contains('주차')]

drop_index = traf_parkingLot_raw[traf_parkingLot_raw['DGM_AR']==0].index
traf_parkingLot_raw = traf_parkingLot_raw.drop(drop_index)

traf_parkingLot_raw['lon']=traf_parkingLot_raw['geometry'].centroid.x  #multipoligon형태여서 중심값 추출
traf_parkingLot_raw['lat']=traf_parkingLot_raw['geometry'].centroid.y
traf_parkingLot_raw['geometry'] = traf_parkingLot_raw.apply(lambda row : Point([row['lon'], row['lat']]), axis=1)
traf_parkingLot_raw['category'] = '주차장'

traf_parkingLot_raw.reset_index(drop=True,inplace=True)

traf_parkingLot_raw['소유주체'] = np.nan
for i in range(len(traf_parkingLot_raw)):
    for j in range(len(owner_land)):
        if traf_parkingLot_raw.geometry[i].within(owner_land.geometry[j]) == True:
            traf_parkingLot_raw['소유주체'][i]=owner_land.소유구분명[j]

traf_parkingLot_raw

,PRESENT_SN,DGM_NM,DGM_AR,DGM_LT,geometry,lon,lat,category
0,46230UQ152PS201202010003,기타 주차장시설,344,174,POINT (127.58425 34.97704),127.584247,34.977039,주차장
1,46230UQ152PS201202010002,기타 주차장시설,483,94,POINT (127.58390 34.97837),127.583898,34.978368,주차장
2,46230UQ152PS201202010109,노외주차장,1612,167,POINT (127.75138 34.94576),127.751378,34.945756,주차장
3,46230UQ152PS201202010106,노외주차장,305,70,POINT (127.70270 34.98894),127.702705,34.988940,주차장
4,46230UQ152PS201202010105,기타 주차장시설,4608,286,POINT (127.68964 34.93745),127.689638,34.937446,주차장
...,...,...,...,...,...,...,...,...
203,46230UQ152PS201202010153,노외주차장,805,127,POINT (127.58994 34.97387),127.589941,34.973871,주차장
204,46230UQ152PS201202010143,기타 주차장시설,1244,144,POINT (127.66659 34.93274),127.666590,34.932737,주차장
205,46230UQ152PS201202010139,노외주차장,2657,208,POINT (127.58461 34.97563),127.584608,34.975629,주차장
206,46230UQ152PS201202010130,노외주차장,1313,142,POINT (127.69850 34.92671),127.698495,34.926713,주차장


# 재서야 주차면수 계산하는거 코드 추가해주라~!~!~! 

In [8]:
traf_parkingLot_raw.to_csv("광양시_도시계획(교통정보)_전처리.csv", index=False, encoding='cp949')

# 4. 건물정보

--> 너무 길어서 얘는 주피터 노트북 따로 만듦! 나중에 어차피 합쳐야함 ㅜㅜ